In [1]:
%%capture
# %pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes datasets trl datasets wandb



In [2]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests


In [3]:
import pandas as pd
from PIL import Image
train_img_path = 'images/'
def train_img_id_str(img_id):
    img_id = str(img_id)
    return train_img_path+'0'*(12-len(img_id))+img_id+'.jpg'

def get_img(image_file):
    image = None
    try:
        image = Image.open(train_img_id_str(image_file)).convert('RGB')
    except:
        image = None
    if image is None:
        return None
    return 1

train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('test.csv')
print(len(train_data), len(valid_data))

print(train_data.columns)

11999 3001
Index(['Unnamed: 0', 'question', 'answer', 'img_id', 'type'], dtype='object')


In [4]:
cnt = 0
import os
for name in train_data['img_id'].tolist():
    if os.path.isfile(train_img_id_str(name)):
        cnt += 1
print(cnt)

3450


In [5]:
from datasets import Dataset

train_data['image'] = train_data['img_id'].apply(get_img)
train_data.dropna(inplace=True)
train_data.drop(['image'], axis=1, inplace=True)
valid_data['image'] = valid_data['img_id'].apply(get_img)
valid_data.dropna(inplace=True)
valid_data.drop(['image'], axis=1, inplace=True)
print(len(train_data), len(valid_data))
train_dataset = Dataset.from_pandas(train_data)
valid_dataset = Dataset.from_pandas(valid_data)

3450 859


In [14]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 171.69 MiB is free. Process 2899651 has 23.52 GiB memory in use. Of the allocated memory 22.14 GiB is allocated by PyTorch, and 1.06 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
image = Image.open(train_img_id_str(train_data['img_id'][0])).convert('RGB')
print(train_data['question'][0], train_data['answer'][0])
print(image)
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": train_data['question'][0]},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=200)

print(processor.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


màu của chiếc bình là gì màu xanh lá
<PIL.Image.Image image mode=RGB size=640x536 at 0x7F0134735B10>
[INST]  
màu của chiếc bình là gì [/INST] Chiếc bình đỏ trong hình tròn có màu của nó là gì? 


In [12]:
torch.cuda.empty_cache()

In [13]:
import wandb
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import torch 

wandb.login(key='e8b5197b26c6afeb71d44e9a9f6fa9c889efb030')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=8, 
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 171.69 MiB is free. Process 2899651 has 23.52 GiB memory in use. Of the allocated memory 21.54 GiB is allocated by PyTorch, and 1.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def collate_fn(examples):
    texts = ["answer " + example["question"] for example in examples]
    labels= [example['answer'] for example in examples]
    images = [Image.open(example["img_id"]).convert("RGB") for example in examples]
    tokens = processor(text=texts, images=images, suffix=labels, return_tensors="pt", padding="longest")
    tokens = tokens.to(torch.bfloat16)
    return tokens

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, Trainer
from peft import TaskType


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=16,
    learning_rate=1e-5,
    eval_strategy="steps",
    eval_steps = 1000,
    save_strategy="steps", # Change to "steps" to save after a certain number of steps
    save_steps=1000, # Save after every 1000 steps
    save_total_limit=5, # Maximum of 5 saves
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    remove_unused_columns=False
    
)

trainer = Trainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset =valid_dataset,
#     peft_config=peft_config,
#     max_seq_length =512,
    data_collator=collate_fn,
#     eval_packing=False,
)

trainer.train()

In [ ]:
peft_model_id="llava-finetune"
trainer.model.save_pretrained(peft_model_id)
processor.save_pretrained(peft_model_id)